In [1]:
from pyquery import PyQuery as pq
from selenium import webdriver
import requests
import pandas as pd
import time,datetime  #時間套件 日期輸出
import random

In [2]:
def fb_Crawler(url):
    #scroll To 50000px
    driver.get(url)
    last_height = 0
    while True:
        box = driver.find_elements_by_css_selector("body")[0]
        driver.execute_script("window.scrollTo(0,{})".format(box.size['height']))
#         print(box.size['height'])
        time.sleep(1)
        time.sleep(0.5+random.random()*1)
        if last_height >= 100000:
#             print("last height", last_height)
            break
        else:
            last_height = int(box.size['height'])

    html = driver.find_element_by_css_selector('*').get_attribute('outerHTML')
    doc = pq(html)
    fb_content_list = my_content(doc)
    return fb_content_list

In [3]:
def my_content(doc):
    my_list2 = []
    for i in doc("._3ccb").items():
        my_list1 = []
        fb_time = (i("._1dwg .timestampContent").text())
        fb_content = (i("._1dwg p").text()[0:10].replace("\n",""))
        if fb_content == "":
            fb_content = ("更新封面照片或其他")
        fb_like = (i("._3vum a ._3dli").text())
        if fb_like == "": #它可能是直播
            fb_like = (i(".UFILikeSentenceText").text().split()[-2])
            if "萬" in (i(".UFILikeSentenceText").text().split()[-1]):
                fb_like += " 萬"
            fb_msg = "video"
            try:
#                 print(i(".UFIShareRow").text())
                fb_share = (i(".UFIShareRow").text().split()[0])
            except:
                continue
            fb_watch = (i("._1t6k .fcg").text())
        else:
            try:
                fb_msg = i("._3vum a._42ft").text().split()[0]
            except:
                fb_msg = "-1"
            try:
                fb_share = i("._3vum a._42ft").text().split()[1]
            except:
                fb_share = "-1"
            fb_watch = "post"
        my_list1 = [fb_time,fb_content,fb_like,fb_msg,fb_share,fb_watch]
        my_list2.append(my_list1)
    return my_list2

In [4]:
# fb_format_data_frame
def fb_format_df(fb_person_list):
    #格式調整
    t3=datetime.datetime.now()
    my_list3 = []
    for i in fb_person_list:
        my_list2 = []
        #時間： 沒有年份補上今年，十、分、秒刪去
        day = i[0].find("日")
        post_date = i[0][0:day+1]
        if "年" not in post_date:
            post_date = str(t3.year) + "年" + post_date
        if "日" not in i[0]:
            post_date = "today"

        my_list2.append(post_date)
        if post_date == "today":
            time_value = time.time()
        else:
            try:
                time_value = datetime.datetime.strptime(post_date,'%Y年%m月%d日').timestamp()
            except:
                continue
        my_list2.append(time_value)

        my_list2.append(i[1])
        #按讚數 去逗點、去萬
        like = i[2].replace(",","")
        if len(like.split()) != 1:
            like = float(like.split()[0])*10000
        my_list2.append(int(like))
        leave_msg = i[3].replace("則留言","").replace(",","")
        if leave_msg == 'video':
            leave_msg = "-1"
        try:
            my_list2.append(int(leave_msg))
        except:
            try:
                leave_msg = float(leave_msg.split()[0])*10000
            except:
                continue
        share = i[4].replace("次分享","").replace(",","")
        try:
            my_list2.append(int(share))
        except:
            my_list2.append(-1)
            
        fb_watch = i[5]
        if fb_watch == 'post':
            fb_watch = "-1"
        if len(fb_watch.split()) != 1:
            try:
                fb_watch = float(fb_watch.split(' ')[0])*10000
            except:
                fb_watch = float(fb_watch.split('\xa0萬')[0])*10000
        my_list2.append(int(fb_watch))
        my_list3.append(my_list2)

    df = pd.DataFrame(my_list3,columns=['時間','時間數值','內容','按讚','留言','分享','觀看次數'])
    return df

In [5]:
url = input("請輸入網址: ")
# url = "https://www.facebook.com/DoctorKoWJ/"

name = input("請輸入存檔名稱: ")
# name = "DoctorKoWJ"

driver = webdriver.Chrome('./chromedriver')

fans_lists=[]
fans_list = fb_Crawler(url)
single_fans_df = fb_format_df(fans_list)
single_fans_df.to_csv('data/fb_{}.csv'.format(name))
driver.close()
print("完成")

請輸入網址: https://www.facebook.com/DoctorKoWJ/
請輸入存檔名稱: DoctorKoWJ
完成


my_input = input()
my_df = pd.read_csv('./data/fb_{}.csv'.format(my_input))
my_df